In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

# CPSC 330 - Applied Machine Learning

## Homework 8: Introduction to Computer vision, Time Series, and Survival Analysis (Lectures 19 to 20) 

**Due date: see the [Apr 07, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points:2}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset and storing it under the `data` folder. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 
rubric={points:4}

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture demo, we had different measurements for each Location. 

We want you to consider this for the avocado prices dataset. For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [9]:
df.sort_values(by = ['Date']).head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,organic,2015,Southeast
51,2015-01-04,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,organic,2015,Chicago
51,2015-01-04,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,organic,2015,HarrisburgScranton
51,2015-01-04,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0,conventional,2015,Pittsburgh
51,2015-01-04,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0,organic,2015,Boise


The printed results above we can see that there are different results in the same date. Categorical features, type has 'organic' and 'conventional' separate measurements; regions has separate measurements too. In the same date, different regions has different price and volume. Now sort by region and date to see the results in the same region and date.

In [10]:
df.sort_values(by = ['region', 'Date']).head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
51,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.0,organic,2015,Albany
50,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
50,2015-01-11,1.77,1182.56,39.00,305.12,0.00,838.44,838.44,0.00,0.0,organic,2015,Albany
49,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany


The printed results above we can see that there are different results in the same date. Categorical features, type has 'organic' and 'conventional' separate measurements; regions has separate measurements too. In the same date, different regions has different price and volume. Now sort by region and date to see the results in the same region and date.

In [11]:
df.sort_values(by = ['region', 'Date']).head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
51,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.0,organic,2015,Albany
50,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
50,2015-01-11,1.77,1182.56,39.00,305.12,0.00,838.44,838.44,0.00,0.0,organic,2015,Albany
49,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany


As we can see, there are also different results in the same Date and the same region. Now we also include 'type'.

In [12]:
df.sort_values(by = ['region', 'type', 'Date']).head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
50,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
49,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
48,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
47,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany


The printed results above shows that the Date is ordered by different timeseries with type in conventional and region in Albany. Both categorical features type and region have separate measurements.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 
rubric={points:4}

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

The spaced measurements are the time difference between sequential date. From 1.1, we explored the region and type are the two categorical feature that have separate measurements, so we will check if the Date differences have equal spaces or not in terms of the different combinations of region and type.

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18249 entries, 0 to 11
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          18249 non-null  datetime64[ns]
 1   AveragePrice  18249 non-null  float64       
 2   Total Volume  18249 non-null  float64       
 3   4046          18249 non-null  float64       
 4   4225          18249 non-null  float64       
 5   4770          18249 non-null  float64       
 6   Total Bags    18249 non-null  float64       
 7   Small Bags    18249 non-null  float64       
 8   Large Bags    18249 non-null  float64       
 9   XLarge Bags   18249 non-null  float64       
 10  type          18249 non-null  object        
 11  year          18249 non-null  int64         
 12  region        18249 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 1.9+ MB


In [14]:
region_type = df.groupby(['region', 'type'])
print(region_type.size())

region               type        
Albany               conventional    169
                     organic         169
Atlanta              conventional    169
                     organic         169
BaltimoreWashington  conventional    169
                                    ... 
TotalUS              organic         169
West                 conventional    169
                     organic         169
WestTexNewMexico     conventional    169
                     organic         166
Length: 108, dtype: int64


We will print out the max and min date difference in each group to see if they are equally spaced.

In [15]:
for (region, types), group in region_type:
    diff = group['Date'].sort_values().diff()
    print(f'({region}, {types}): {diff.max()}, {diff.min()}')

(Albany, conventional): 7 days 00:00:00, 7 days 00:00:00
(Albany, organic): 7 days 00:00:00, 7 days 00:00:00
(Atlanta, conventional): 7 days 00:00:00, 7 days 00:00:00
(Atlanta, organic): 7 days 00:00:00, 7 days 00:00:00
(BaltimoreWashington, conventional): 7 days 00:00:00, 7 days 00:00:00
(BaltimoreWashington, organic): 7 days 00:00:00, 7 days 00:00:00
(Boise, conventional): 7 days 00:00:00, 7 days 00:00:00
(Boise, organic): 7 days 00:00:00, 7 days 00:00:00
(Boston, conventional): 7 days 00:00:00, 7 days 00:00:00
(Boston, organic): 7 days 00:00:00, 7 days 00:00:00
(BuffaloRochester, conventional): 7 days 00:00:00, 7 days 00:00:00
(BuffaloRochester, organic): 7 days 00:00:00, 7 days 00:00:00
(California, conventional): 7 days 00:00:00, 7 days 00:00:00
(California, organic): 7 days 00:00:00, 7 days 00:00:00
(Charlotte, conventional): 7 days 00:00:00, 7 days 00:00:00
(Charlotte, organic): 7 days 00:00:00, 7 days 00:00:00
(Chicago, conventional): 7 days 00:00:00, 7 days 00:00:00
(Chicago, 

The results are showing that almost all spaces between sequential Dates are equally separated 7 days, except (WestTexNewMexico, organic) has two Dates with 21 days difference. Combining with the df.info() and .size(), we can see there are no missing values across all features, the group (WestTexNewMexico, organic) have 3 measurements less than other groups.

In [16]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 
rubric={points:4}

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [17]:
df['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2024W2/tree/main/lectures), with some improvements.

In [18]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [19]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [20]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [21]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 `AveragePrice` baseline 
rubric={points}

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [22]:
train_r2 = r2_score(df_train['AveragePriceNextWeek'], df_train['AveragePrice'])
train_r2

0.8285800937261841

In [23]:
test_r2 = r2_score(df_test['AveragePriceNextWeek'], df_test['AveragePrice'])
test_r2

0.7631780188583048

In [24]:
...

Ellipsis

In [25]:
...

Ellipsis

In [26]:
assert not train_r2 is None, "Are you using the correct variable name?"
assert not test_r2 is None, "Are you using the correct variable name?"
assert sha1(str(round(train_r2, 3)).encode('utf8')).hexdigest() == 'b1136fe2a8918904393ab6f40bfb3f38eac5fc39', "Your training score is not correct. Are you using the right features?"
assert sha1(str(round(test_r2, 3)).encode('utf8')).hexdigest() == 'cc24d9a9b567b491a56b42f7adc582f2eefa5907', "Your test score is not correct. Are you using the right features?"

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price
rubric={points:10}

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 10

**Approach 1: encoding time as a number, create a column of days since the earliest date (2015-01-04) in the dataset.**

In [27]:
first_day = df_train['Date'].min()
first_day

Timestamp('2015-01-04 00:00:00')

In [28]:
df_train = df_train.assign(
    Days_since = df_train['Date'].apply(lambda x: (x - first_day).days)
)

df_test = df_test.assign(
    Days_since = df_test['Date'].apply(lambda x: (x - first_day).days)
)

df_train.sort_values(by='Date').head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek,Days_since
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24,0
13858,2015-01-04,0.94,461607.33,244152.26,165299.33,15302.75,36852.99,30884.29,5595.00,373.7,conventional,2015,SanDiego,0.82,0
1352,2015-01-04,1.02,491738.00,7193.87,396752.18,128.82,87663.13,87406.84,256.29,0.0,conventional,2015,Boston,1.10,0
13689,2015-01-04,1.33,9213.49,3727.52,4327.52,0.00,1158.45,1158.45,0.00,0.0,organic,2015,Sacramento,1.27,0
13520,2015-01-04,1.05,430138.88,110693.69,270107.61,9737.50,39600.08,39600.08,0.00,0.0,conventional,2015,Sacramento,1.09,0


Apply preprocessing on other features

In [29]:
numeric_feats = [
    'Total Volume',
    '4046',
    '4225',
    '4770',
    'Total Bags',
    'Small Bags',
    'Large Bags',
    'XLarge Bags',
]

categorical_feats = ['region']
ordinal_feats = ['year']
binary_feats = ['type']
passthrough_feats = ['AveragePrice', 'Days_since']
drop_feats = ['Date', 'AveragePriceNextWeek']

In [30]:
year_levels = ['2015', '2016', '2017', '2018']

In [31]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)
ordinal_transformer = OrdinalEncoder(categories = [year_levels], dtype = int)
binary_transformer = OneHotEncoder(handle_unknown = 'ignore', drop = 'if_binary', dtype = int)

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_feats),
    (categorical_transformer, categorical_feats),
    (ordinal_transformer, ordinal_feats),
    (binary_transformer, binary_feats),
    ('drop', drop_feats),
    ('passthrough', passthrough_feats),
)
preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['Total Volume', '4046', '4225', '4770',
                                  'Total Bags', 'Small Bags', 'Large Bags',
                                  'XLarge Bags']),
                                ('onehotencoder-1',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['region']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['2015', '2016',
                                                             '2017', '2018']],
                                                dtype=<class 'int'>),
                                 ['year']),
                                ('onehotencoder-2',
                                 OneHotEncoder(drop='if_binary',
                                               dtype=<class 'int'>,
                                               handle_unknown='ignore'),
                                 ['type']),
                                ('drop', 'drop',
                                 ['Date', 'AveragePriceNextWeek']),
                                ('passthrough', 'passthrough',
                                 ['AveragePrice', 'Days_since'])])

In [32]:
preprocessor.fit(df_train)
column_names = numeric_feats + list(
    preprocessor.named_transformers_['onehotencoder-1'].get_feature_names_out(
        categorical_feats
    )) + list(
        preprocessor.named_transformers_['ordinalencoder'].get_feature_names_out(
            ordinal_feats
        )) + list(
            preprocessor.named_transformers_['onehotencoder-2'].get_feature_names_out(
                binary_feats
            )) + passthrough_feats
column_names

['Total Volume',
 '4046',
 '4225',
 '4770',
 'Total Bags',
 'Small Bags',
 'Large Bags',
 'XLarge Bags',
 'region_Albany',
 'region_Atlanta',
 'region_BaltimoreWashington',
 'region_Boise',
 'region_Boston',
 'region_BuffaloRochester',
 'region_California',
 'region_Charlotte',
 'region_Chicago',
 'region_CincinnatiDayton',
 'region_Columbus',
 'region_DallasFtWorth',
 'region_Denver',
 'region_Detroit',
 'region_GrandRapids',
 'region_GreatLakes',
 'region_HarrisburgScranton',
 'region_HartfordSpringfield',
 'region_Houston',
 'region_Indianapolis',
 'region_Jacksonville',
 'region_LasVegas',
 'region_LosAngeles',
 'region_Louisville',
 'region_MiamiFtLauderdale',
 'region_Midsouth',
 'region_Nashville',
 'region_NewOrleansMobile',
 'region_NewYork',
 'region_Northeast',
 'region_NorthernNewEngland',
 'region_Orlando',
 'region_Philadelphia',
 'region_PhoenixTucson',
 'region_Pittsburgh',
 'region_Plains',
 'region_Portland',
 'region_RaleighGreensboro',
 'region_RichmondNorfolk',
 'r

In [33]:
df_train.shape

(15441, 15)

In [34]:
preprocessor.transform(df_train).shape

(15441, 66)

In [35]:
X_train_enc = pd.DataFrame(preprocessor.transform(df_train), columns = column_names)
X_test_enc = pd.DataFrame(preprocessor.transform(df_test), columns = column_names)
X_train_enc

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,region_Albany,region_Atlanta,...,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico,year,type_organic,AveragePrice,Days_since
0,-0.234535,-0.229503,-0.222203,-0.214954,-0.232206,-0.229907,-0.223154,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.22,0.0
1,-0.234440,-0.230948,-0.219448,-0.214272,-0.233587,-0.231513,-0.223789,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.24,7.0
2,-0.233469,-0.231018,-0.219530,-0.214196,-0.229850,-0.226469,-0.224325,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.17,14.0
3,-0.233283,-0.230996,-0.218170,-0.213945,-0.230999,-0.228629,-0.222193,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.06,21.0
4,-0.225747,-0.230668,-0.196131,-0.213811,-0.232627,-0.229930,-0.224856,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15436,-0.241780,-0.229664,-0.242429,-0.215395,-0.232066,-0.229029,-0.225402,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.50,966.0
15437,-0.244264,-0.231008,-0.242114,-0.215395,-0.239735,-0.239057,-0.225344,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.39,973.0
15438,-0.243034,-0.230503,-0.241696,-0.215395,-0.236468,-0.234745,-0.225499,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.38,980.0
15439,-0.243442,-0.230278,-0.243130,-0.215395,-0.236393,-0.234638,-0.225529,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.36,987.0


In [36]:
y_train = df_train['AveragePriceNextWeek']
y_test = df_test['AveragePriceNextWeek']
y_train

0        1.24
1        1.17
2        1.06
3        0.99
4        0.99
         ... 
18218    2.39
18219    2.38
18220    2.36
18221    2.26
18222    2.37
Name: AveragePriceNextWeek, Length: 15441, dtype: float64

In [37]:
#train ridge
model = Ridge()
model.fit(X_train_enc, y_train)
train_pred = model.predict(X_train_enc)
test_pred = model.predict(X_test_enc)
print(train_pred)
print(test_pred)

[1.20485182 1.2214329  1.16659252 ... 2.23257341 2.21743791 2.13897308]
[1.65040155 1.72237396 1.62016906 ... 1.5707403  1.587188   1.58803667]


In [38]:
#train score 
approach1_train_r2 = r2_score(y_train, train_pred)
approach1_train_r2

0.8456630861031358

In [39]:
#test score
approach1_test_r2 = r2_score(y_test, test_pred)
approach1_test_r2

0.7838671502482812

The approach 1 got train r2 score 0.8456, slightly better than baseline train score 0.8285; test r2 score is about 0.7838, which is slightly better than our baseline test score 0.76.

**Approach 2:Onehot encoding for the month**

In [40]:
#add months and sort by days 
df_train = df_train.assign(
    Month = df_train['Date'].apply(lambda x: x.month_name())
)
df_test = df_test.assign(
    Month = df_train['Date'].apply(lambda x: x.month_name())
)

df_train[['Date', 'Month']].sort_values(by='Date')

,Date,Month
0,2015-01-04,January
13858,2015-01-04,January
1352,2015-01-04,January
13689,2015-01-04,January
13520,2015-01-04,January
...,...,...
9775,2017-09-24,September
9606,2017-09-24,September
9437,2017-09-24,September
6057,2017-09-24,September


In [41]:
#preprocesssor setup
categorical_feats = categorical_feats + ['Month']
passthrough_feats = ['AveragePrice']
drop_feats = drop_feats + ['Days_since']
preprocessor = make_column_transformer(
    (numeric_transformer, numeric_feats),
    (categorical_transformer, categorical_feats),
    (ordinal_transformer, ordinal_feats),
    (binary_transformer, binary_feats),
    ('drop', drop_feats),
    ('passthrough', passthrough_feats),
)
preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['Total Volume', '4046', '4225', '4770',
                                  'Total Bags', 'Small Bags', 'Large Bags',
                                  'XLarge Bags']),
                                ('onehotencoder-1',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['region', 'Month']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['2015', '2016',
                                                             '2017', '2018']],
                                                dtype=<class 'int'>),
                                 ['year']),
                                ('onehotencoder-2',
                                 OneHotEncoder(drop='if_binary',
                                               dtype=<class 'int'>,
                                               handle_unknown='ignore'),
                                 ['type']),
                                ('drop', 'drop',
                                 ['Date', 'AveragePriceNextWeek',
                                  'Days_since']),
                                ('passthrough', 'passthrough',
                                 ['AveragePrice'])])

In [42]:
preprocessor.fit(df_train)
column_names = numeric_feats + list(
    preprocessor.named_transformers_['onehotencoder-1'].get_feature_names_out(
        categorical_feats
    )) + list(
        preprocessor.named_transformers_['ordinalencoder'].get_feature_names_out(
            ordinal_feats
        )) + list(
            preprocessor.named_transformers_['onehotencoder-2'].get_feature_names_out(
                binary_feats
            )) + passthrough_feats
column_names

['Total Volume',
 '4046',
 '4225',
 '4770',
 'Total Bags',
 'Small Bags',
 'Large Bags',
 'XLarge Bags',
 'region_Albany',
 'region_Atlanta',
 'region_BaltimoreWashington',
 'region_Boise',
 'region_Boston',
 'region_BuffaloRochester',
 'region_California',
 'region_Charlotte',
 'region_Chicago',
 'region_CincinnatiDayton',
 'region_Columbus',
 'region_DallasFtWorth',
 'region_Denver',
 'region_Detroit',
 'region_GrandRapids',
 'region_GreatLakes',
 'region_HarrisburgScranton',
 'region_HartfordSpringfield',
 'region_Houston',
 'region_Indianapolis',
 'region_Jacksonville',
 'region_LasVegas',
 'region_LosAngeles',
 'region_Louisville',
 'region_MiamiFtLauderdale',
 'region_Midsouth',
 'region_Nashville',
 'region_NewOrleansMobile',
 'region_NewYork',
 'region_Northeast',
 'region_NorthernNewEngland',
 'region_Orlando',
 'region_Philadelphia',
 'region_PhoenixTucson',
 'region_Pittsburgh',
 'region_Plains',
 'region_Portland',
 'region_RaleighGreensboro',
 'region_RichmondNorfolk',
 'r

In [43]:
X_train_enc = pd.DataFrame(preprocessor.transform(df_train), columns = column_names)
X_test_enc = pd.DataFrame(preprocessor.transform(df_test), columns = column_names)
X_train_enc

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,region_Albany,region_Atlanta,...,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September,year,type_organic,AveragePrice
0,-0.234535,-0.229503,-0.222203,-0.214954,-0.232206,-0.229907,-0.223154,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.22
1,-0.234440,-0.230948,-0.219448,-0.214272,-0.233587,-0.231513,-0.223789,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.24
2,-0.233469,-0.231018,-0.219530,-0.214196,-0.229850,-0.226469,-0.224325,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.17
3,-0.233283,-0.230996,-0.218170,-0.213945,-0.230999,-0.228629,-0.222193,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.06
4,-0.225747,-0.230668,-0.196131,-0.213811,-0.232627,-0.229930,-0.224856,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15436,-0.241780,-0.229664,-0.242429,-0.215395,-0.232066,-0.229029,-0.225402,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.50
15437,-0.244264,-0.231008,-0.242114,-0.215395,-0.239735,-0.239057,-0.225344,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.39
15438,-0.243034,-0.230503,-0.241696,-0.215395,-0.236468,-0.234745,-0.225499,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.38
15439,-0.243442,-0.230278,-0.243130,-0.215395,-0.236393,-0.234638,-0.225529,-0.172063,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.36


In [44]:
y_train = df_train['AveragePriceNextWeek']
y_test = df_test['AveragePriceNextWeek']
y_train

0        1.24
1        1.17
2        1.06
3        0.99
4        0.99
         ... 
18218    2.39
18219    2.38
18220    2.36
18221    2.26
18222    2.37
Name: AveragePriceNextWeek, Length: 15441, dtype: float64

In [45]:
model = Ridge()
model.fit(X_train_enc, y_train)
train_pred = model.predict(X_train_enc)
test_pred = model.predict(X_test_enc)
print(train_pred)
print(test_pred)

[1.17829281 1.19375676 1.13963585 ... 2.25491833 2.2394789  2.16233196]
[1.62911054 1.69854324 1.59825313 ... 1.57522881 1.59056956 1.59069568]


In [46]:
approach2_train_r2 = r2_score(y_train, train_pred)
approach2_train_r2

0.8493687923226023

In [47]:
approach2_test_r2 = r2_score(y_test, test_pred)
approach2_test_r2

0.7891284204406306

The approach 2 got train r2 score 0.849, slightly better than both baseline and approach 1 train score; test r2 score is about 0.789, which is slightly better than both our baseline and approach 1 test score.

**Approach 3:onehot encoding season**

In [48]:
def get_season(month):
    # remember this is Australia
    WINTER_MONTHS = ["June", "July", "August"] 
    AUTUMN_MONTHS = ["March", "April", "May"]
    SUMMER_MONTHS = ["December", "January", "February"]
    SPRING_MONTHS = ["September", "October", "November"]
    if month in WINTER_MONTHS:
        return "Winter"
    elif month in AUTUMN_MONTHS:
        return "Autumn"
    elif month in SUMMER_MONTHS:
        return "Summer"
    else:
        return "Fall"

In [49]:
df_train = df_train.assign(Season=df_train["Month"].apply(get_season))
df_test = df_test.assign(Season=df_test["Month"].apply(get_season))
df_train

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek,Days_since,Month,Season
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24,0,January,Summer
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17,7,January,Summer
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06,14,January,Summer
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99,21,January,Summer
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99,28,February,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18218,2017-08-27,2.50,16137.93,2616.96,3672.96,0.00,9848.01,9816.58,31.43,0.0,organic,2017,WestTexNewMexico,2.39,966,August,Winter
18219,2017-09-03,2.39,7657.47,927.27,4056.73,0.00,2673.47,2629.18,44.29,0.0,organic,2017,WestTexNewMexico,2.38,973,September,Fall
18220,2017-09-10,2.38,11857.31,1562.10,4565.41,0.00,5729.80,5719.96,9.84,0.0,organic,2017,WestTexNewMexico,2.36,980,September,Fall
18221,2017-09-17,2.36,10464.29,1845.14,2819.17,0.00,5799.98,5796.65,3.33,0.0,organic,2017,WestTexNewMexico,2.26,987,September,Fall


In [50]:
categorical_feats =  ['region', 'Season']
drop_feats = drop_feats + ['Days_since'] + ['Month']
preprocessor = make_column_transformer(
    (numeric_transformer, numeric_feats),
    (categorical_transformer, categorical_feats),
    (ordinal_transformer, ordinal_feats),
    (binary_transformer, binary_feats),
    ('drop', drop_feats),
    ('passthrough', passthrough_feats),
)
preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['Total Volume', '4046', '4225', '4770',
                                  'Total Bags', 'Small Bags', 'Large Bags',
                                  'XLarge Bags']),
                                ('onehotencoder-1',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['region', 'Season']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['2015', '2016',
                                                             '2017', '2018']],
                                                dtype=<class 'int'>),
                                 ['year']),
                                ('onehotencoder-2',
                                 OneHotEncoder(drop='if_binary',
                                               dtype=<class 'int'>,
                                               handle_unknown='ignore'),
                                 ['type']),
                                ('drop', 'drop',
                                 ['Date', 'AveragePriceNextWeek', 'Days_since',
                                  'Days_since', 'Month']),
                                ('passthrough', 'passthrough',
                                 ['AveragePrice'])])

In [51]:
preprocessor.fit(df_train)
column_names = numeric_feats + list(
    preprocessor.named_transformers_['onehotencoder-1'].get_feature_names_out(
        categorical_feats
    )) + list(
        preprocessor.named_transformers_['ordinalencoder'].get_feature_names_out(
            ordinal_feats
        )) + list(
            preprocessor.named_transformers_['onehotencoder-2'].get_feature_names_out(
                binary_feats
            )) + passthrough_feats
column_names

['Total Volume',
 '4046',
 '4225',
 '4770',
 'Total Bags',
 'Small Bags',
 'Large Bags',
 'XLarge Bags',
 'region_Albany',
 'region_Atlanta',
 'region_BaltimoreWashington',
 'region_Boise',
 'region_Boston',
 'region_BuffaloRochester',
 'region_California',
 'region_Charlotte',
 'region_Chicago',
 'region_CincinnatiDayton',
 'region_Columbus',
 'region_DallasFtWorth',
 'region_Denver',
 'region_Detroit',
 'region_GrandRapids',
 'region_GreatLakes',
 'region_HarrisburgScranton',
 'region_HartfordSpringfield',
 'region_Houston',
 'region_Indianapolis',
 'region_Jacksonville',
 'region_LasVegas',
 'region_LosAngeles',
 'region_Louisville',
 'region_MiamiFtLauderdale',
 'region_Midsouth',
 'region_Nashville',
 'region_NewOrleansMobile',
 'region_NewYork',
 'region_Northeast',
 'region_NorthernNewEngland',
 'region_Orlando',
 'region_Philadelphia',
 'region_PhoenixTucson',
 'region_Pittsburgh',
 'region_Plains',
 'region_Portland',
 'region_RaleighGreensboro',
 'region_RichmondNorfolk',
 'r

In [52]:
X_train_enc = pd.DataFrame(preprocessor.transform(df_train), columns = column_names)
X_test_enc = pd.DataFrame(preprocessor.transform(df_test), columns = column_names)
X_train_enc

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,region_Albany,region_Atlanta,...,region_TotalUS,region_West,region_WestTexNewMexico,Season_Autumn,Season_Fall,Season_Summer,Season_Winter,year,type_organic,AveragePrice
0,-0.234535,-0.229503,-0.222203,-0.214954,-0.232206,-0.229907,-0.223154,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.22
1,-0.234440,-0.230948,-0.219448,-0.214272,-0.233587,-0.231513,-0.223789,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.24
2,-0.233469,-0.231018,-0.219530,-0.214196,-0.229850,-0.226469,-0.224325,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.17
3,-0.233283,-0.230996,-0.218170,-0.213945,-0.230999,-0.228629,-0.222193,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.06
4,-0.225747,-0.230668,-0.196131,-0.213811,-0.232627,-0.229930,-0.224856,-0.172063,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15436,-0.241780,-0.229664,-0.242429,-0.215395,-0.232066,-0.229029,-0.225402,-0.172063,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,2.50
15437,-0.244264,-0.231008,-0.242114,-0.215395,-0.239735,-0.239057,-0.225344,-0.172063,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,2.39
15438,-0.243034,-0.230503,-0.241696,-0.215395,-0.236468,-0.234745,-0.225499,-0.172063,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,2.38
15439,-0.243442,-0.230278,-0.243130,-0.215395,-0.236393,-0.234638,-0.225529,-0.172063,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,2.36


In [53]:
y_train = df_train['AveragePriceNextWeek']
y_test = df_test['AveragePriceNextWeek']
y_train

0        1.24
1        1.17
2        1.06
3        0.99
4        0.99
         ... 
18218    2.39
18219    2.38
18220    2.36
18221    2.26
18222    2.37
Name: AveragePriceNextWeek, Length: 15441, dtype: float64

In [54]:
model = Ridge()
model.fit(X_train_enc, y_train)
train_pred = model.predict(X_train_enc)
test_pred = model.predict(X_test_enc)
print(train_pred)
print(test_pred)

[1.19123738 1.20680349 1.15233077 ... 2.2299574  2.21441784 2.1367773 ]
[1.65349556 1.72336669 1.62245026 ... 1.59971509 1.61515006 1.61528279]


In [55]:
approach3_train_r2 = r2_score(y_train, train_pred)
approach3_train_r2

0.8475540530429727

In [56]:
approach3_test_r2 = r2_score(y_test, test_pred)
approach3_test_r2

0.7680705839522307

The approach 3 is almost the same as the scores we got in base line.

Overall, the approach 2: one hot encoding month performs the best scores.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Short answer questions

<!-- BEGIN QUESTION -->

### 2.1 Time series

rubric={points:6}

The following questions pertain to Lecture 20 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.
3. When studying time series modeling, we explored several ways to encode date information as a feature for the citibike dataset. When we used time of day as a numeric feature, the Ridge model was not able to capture the periodic pattern. Why? How did we tackle this problem? Briefly explain.

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 6

1. A restaurant's ordering system keeps detailed records of every order.

2. If we try to create lagged features (i.e., values from previous time steps), it won’t work well when the orders are \textbf{not evenly spaced in time}, because lagged features assume data comes at regular intervals.

On the other hand, if we \textbf{encode the date} (for example, extract hour, day, or month), this still works even if the time intervals between orders are uneven.

3.The training prediction was good but the test just predict a horizontal line because it can not capture periodic behaviour of the data that extrapolate to feature ranges outside the training data. We tackled problem by adding hours 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Computer vision 
rubric={points:6}

The following questions pertain to Lecture 19 on multiclass classification and introduction to computer vision. 

1. How many parameters (coefficients and intercepts) will `sklearn`’s `LogisticRegression()` model learn for a four-class classification problem, assuming that you have 10 features? Briefly explain your answer.
2. In Lecture 19, we briefly discussed how neural networks are sort of like `sklearn`'s pipelines, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?
3. Imagine that you have a small dataset with ~1000 images containing pictures and names of 50 different Computer Science faculty members from UBC. Your goal is to develop a reasonably accurate multi-class classification model for this task. Describe which model/technique you would use and briefly justify your choice in one to three sentences.

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 6

1. 44 parameters — one set of 10 weights + 1 intercept per class, for 4 classes.
2. Transfer learning works because we can reuse earlier layers of a neural network as fixed feature extractors, and only retrain the final layer for the new task.
3. Use a pre-trained CNN (e.g., ResNet) and fine-tune it on your dataset. This approach works well when you have a small dataset but need strong visual features.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 2.3 Survival analysis
<hr>

rubric={points:6}

The following questions pertain to Lecture 21 on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer? Briefly explain your answer. 
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.3
    
</div>

_Points:_ 6

1. The problem ignored time, the state of 'churned' or 'not churned' might be changing over time, the 'not churned' people might be 'churned' afterwards, this may cause wrong labeling of those customers.
2. We don't have enough information.
3. A flat customer's survival function indicates that this period has low customer churned risk.

<!-- END QUESTION -->

<br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)